In [3]:
import copy
from collections import deque


In [5]:
class Sudoku_AI_solver:
    def __init__(self, board):
        self.board = board
        self.domains = {
            (i, j): [self.board[i][j]] if self.board[i][j] != 0 else list(range(1, 10))
            for i in range(9) for j in range(9)
        }

    def enforce_node_consistency(self):
        """Remove any values from a cell's domain that violate Sudoku rules."""
        for (i, j), values in self.domains.items():
            if self.board[i][j] != 0:
                self.domains[(i, j)] = [self.board[i][j]]

    def revise(self, x, y):
        """Make variable x arc consistent with variable y."""
        revised = False
        for value in self.domains[x][:]:
            if all(value == other_value for other_value in self.domains[y]):
                self.domains[x].remove(value)
                revised = True
        return revised

    def ac3(self):
        """Use the AC-3 algorithm to make the puzzle arc consistent."""
        queue = deque((x, y) for x in self.domains for y in self.neighbors(x))
        while queue:
            x, y = queue.popleft()
            if self.revise(x, y):
                if not self.domains[x]:
                    return False
                for z in self.neighbors(x) - {y}:
                    queue.append((z, x))
        return True

    def neighbors(self, cell):
        """Return a set of neighbors for a given cell."""
        i, j = cell
        row = {(i, y) for y in range(9)}
        col = {(x, j) for x in range(9)}
        box = {
            (x, y)
            for x in range(i // 3 * 3, i // 3 * 3 + 3)
            for y in range(j // 3 * 3, j // 3 * 3 + 3)
        }
        return (row | col | box) - {cell}

    def assignment_complete(self, assignment):
        """Check if every cell has been assigned a value."""
        return all(len(values) == 1 for values in assignment.values())

    def consistent(self, assignment):
        """Check if the assignment is consistent."""
        for cell, values in assignment.items():
            if len(values) > 1:
                continue
            for neighbor in self.neighbors(cell):
                if len(assignment[neighbor]) == 1 and assignment[neighbor] == values:
                    return False
        return True

    def order_domain_values(self, var, assignment):
        """Order values in the domain of var by least constraining value."""
        return sorted(self.domains[var], key=lambda value: self.count_conflicts(var, value))

    def count_conflicts(self, var, value):
        """Count the number of conflicts that assigning value to var would produce."""
        return sum(
            1
            for neighbor in self.neighbors(var)
            if value in self.domains[neighbor]
        )

    def select_unassigned_variable(self, assignment):
        """Choose the next variable to assign using MRV and Degree heuristics."""
        unassigned = [v for v in assignment if len(assignment[v]) > 1]
        return min(unassigned, key=lambda var: (len(assignment[var]), -len(self.neighbors(var))))

    def backtrack(self, assignment):
        """Perform backtracking search to solve the puzzle."""
        if self.assignment_complete(assignment):
            return assignment

        var = self.select_unassigned_variable(assignment)
        for value in self.order_domain_values(var, assignment):
            new_assignment = copy.deepcopy(assignment)
            new_assignment[var] = [value]
            if self.consistent(new_assignment):
                result = self.backtrack(new_assignment)
                if result:
                    return result
        return None

    def solve(self):
        """Solve the Sudoku puzzle."""
        self.enforce_node_consistency()
        if not self.ac3():
            return None
        return self.backtrack(self.domains)


In [7]:
def parse_board(filename):
    """Parse a Sudoku board from a file."""
    with open(filename, "r") as file:
        return [list(map(int, line.strip().split())) for line in file if line.strip()]

def print_board(board):
    """Print a Sudoku board."""
    for i, row in enumerate(board):
        if i % 3 == 0 and i > 0:
            print("-" * 21)
        print(" ".join(str(cell) if cell != 0 else "." for cell in row))

def solve_and_display(filename):
    """Solve a Sudoku puzzle and display the results."""
    board = parse_board(filename)
    print("Initial Board:")
    print_board(board)

    solver = Sudoku_AI_solver(board)
    solution = solver.solve()

    if solution:
        print("\nSolved Board:")
        solved_board = [[solution[(i, j)][0] for j in range(9)] for i in range(9)]
        print_board(solved_board)
    else:
        print("No solution found.")


In [9]:
import os
print(os.getcwd())


C:\Users\shing


In [11]:
solve_and_display(r"C:\Users\shing\Puzzles\sudoku_easy.txt.txt")

Initial Board:
5 3 . . 7 . . . .
6 . . 1 9 5 . . .
. 9 8 . . . . 6 .
---------------------
8 . . . 6 . . . 3
4 . . 8 . 3 . . 1
7 . . . 2 . . . 6
---------------------
. 6 . . . . 2 8 .
. . . 4 1 9 . . 5
. . . . 8 . . 7 9

Solved Board:
5 3 4 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
---------------------
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
---------------------
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
3 4 5 2 8 6 1 7 9


In [13]:
solve_and_display(r"C:\Users\shing\Puzzles\sudoku_medium.txt.txt")

Initial Board:
. . . 6 . . 4 . .
7 . . . . 3 6 . .
. . . . 9 1 . 8 .
---------------------
. . . . . . . . .
. 5 . 1 8 . . . 3
. . . 3 . 6 . 4 5
---------------------
. 4 . 2 . . . 6 .
9 . 3 . . . . . .
. 2 . . . . 1 . .

Solved Board:
5 8 1 6 7 2 4 3 9
7 9 2 8 4 3 6 5 1
3 6 4 5 9 1 7 8 2
---------------------
4 3 8 9 5 7 2 1 6
2 5 6 1 8 4 9 7 3
1 7 9 3 2 6 8 4 5
---------------------
8 4 5 2 1 9 3 6 7
9 1 3 7 6 8 5 2 4
6 2 7 4 3 5 1 9 8


In [14]:
solve_and_display(r"C:\Users\shing\Puzzles\sudoku_easy.txt.txt")

Initial Board:
5 3 . . 7 . . . .
6 . . 1 9 5 . . .
. 9 8 . . . . 6 .
---------------------
8 . . . 6 . . . 3
4 . . 8 . 3 . . 1
7 . . . 2 . . . 6
---------------------
. 6 . . . . 2 8 .
. . . 4 1 9 . . 5
. . . . 8 . . 7 9

Solved Board:
5 3 4 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
---------------------
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
---------------------
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
3 4 5 2 8 6 1 7 9


In [15]:
solve_and_display(r"C:\Users\shing\Puzzles\sudoku_hard.txt.txt")

Initial Board:
. . . 6 . . 4 . .
7 . . . . 3 6 . .
. . . . 9 1 . 8 .
---------------------
. . . . . . . . .
. 5 . 1 8 . . . 3
. . . 3 . 6 . 4 5
---------------------
. 4 . 2 . . . 6 .
9 . 3 . . . . . .
. 2 . . . . 1 . .

Solved Board:
5 8 1 6 7 2 4 3 9
7 9 2 8 4 3 6 5 1
3 6 4 5 9 1 7 8 2
---------------------
4 3 8 9 5 7 2 1 6
2 5 6 1 8 4 9 7 3
1 7 9 3 2 6 8 4 5
---------------------
8 4 5 2 1 9 3 6 7
9 1 3 7 6 8 5 2 4
6 2 7 4 3 5 1 9 8
